In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'


In [2]:

train = pd.read_csv('D:/Studies/Machine Learning/HackerEarth/1st/Dataset/Train.csv')
test = pd.read_csv('D:/Studies/Machine Learning/HackerEarth/1st/Dataset/Test.csv')
submission = pd.read_csv('D:/Studies/Machine Learning/HackerEarth/1st/Dataset/sample_submission.csv')

emp_id = test.Employee_ID

In [3]:
def print_shape():
    print(train.shape)
    print(test.shape)


In [4]:
def find_na():
    print(train.isnull().sum())
    print("_____________________________")
    print(test.isnull().sum())
    print("_____________________________")

In [5]:
# filling the missing age value with 22 +  year of service taking them as first job in this company

train['Age'] = train['Age'].fillna(22+ train['Time_of_service']) 
test['Age'] = test['Age'].fillna(22+ test['Time_of_service']) 



train['Age'].fillna((train['Age']).mean(),inplace = True)
test['Age'].fillna((train['Age']).mean(),inplace = True)



train['Time_of_service'] = train['Time_of_service'].fillna(train['Age'] - 22.0) 
test['Time_of_service'] = test['Time_of_service'].fillna(test['Age'] - 22.0)


diction = {'1.0':'1.0','2.0':'1.0','3.0':'2.0','4.0':'2.0','5.0':'3.0',
          '6.0':'3.0','7.0':'4.0','8.0':'4.0','9.0':'5.0','10.0':'5.0',}
fun = {'1.0':'2.0','2.0':'4.0','3.0':'6.0','4.0':'8.0','5.0':'10.0',}

train['Work_Life_balance'] = train['Work_Life_balance'].fillna(train['Pay_Scale'].map(diction))
train['Work_Life_balance'].fillna(train['Work_Life_balance'].mean(),inplace=True)
test['Work_Life_balance'] = test['Work_Life_balance'].fillna(test['Pay_Scale'].map(diction))
test['Work_Life_balance'].fillna(test['Work_Life_balance'].mean(),inplace=True)

train['Pay_Scale'].fillna(train['Work_Life_balance'].mean(),inplace=True)
test['Pay_Scale'].fillna(test['Work_Life_balance'].mean(),inplace=True)

train.Gender[train.Gender == 'M'] = 0
train.Gender[train.Gender == 'F'] = 1
test.Gender[test.Gender == 'M'] = 0
test.Gender[test.Gender == 'F'] = 1

train.Relationship_Status[train.Relationship_Status == 'Married'] =0 
test.Relationship_Status[test.Relationship_Status =='Married'] =0
train.Relationship_Status[train.Relationship_Status == 'Single'] =1 
test.Relationship_Status[test.Relationship_Status =='Single'] =1

train.head(2)
test.head(2)

train.drop(['VAR1','VAR2','VAR3','VAR4','VAR5','VAR6','VAR7','Employee_ID'],axis=1,inplace=True)
test.drop(['VAR1','VAR2','VAR3','VAR4','VAR5','VAR6','VAR7','Employee_ID'],axis=1,inplace=True)


train.Compensation_and_Benefits[train.Compensation_and_Benefits == 'type0'] = 0
train.Compensation_and_Benefits[train.Compensation_and_Benefits == 'type1'] = 1
train.Compensation_and_Benefits[train.Compensation_and_Benefits == 'type2'] = 2
train.Compensation_and_Benefits[train.Compensation_and_Benefits == 'type3'] = 3
train.Compensation_and_Benefits[train.Compensation_and_Benefits == 'type4'] = 4
test.Compensation_and_Benefits[test.Compensation_and_Benefits == 'type0'] = 0
test.Compensation_and_Benefits[test.Compensation_and_Benefits == 'type1'] = 1
test.Compensation_and_Benefits[test.Compensation_and_Benefits == 'type2'] = 2
test.Compensation_and_Benefits[test.Compensation_and_Benefits == 'type3'] = 3
test.Compensation_and_Benefits[test.Compensation_and_Benefits == 'type4'] = 4



In [6]:
home = {'Franklin':0,'Springfield':1,'Clinton':2,'Lebanon':3,'Washington':4}

train['Hometown'] = train['Hometown'].map(home)
test['Hometown'] = test['Hometown'].map(home)

In [7]:
uni = {'IT':0, 'Logistics':1, 'Quality':2, 'Human Resource Management':3, 'Purchasing':4,
 'Sales':5, 'Production':6, 'Operarions':7, 'Accounting and Finance':8, 'Marketing':9,
 'R&D':10, 'Security':11}

train['Unit'] = train['Unit'].map(uni)
test['Unit'] = test['Unit'].map(uni)

In [8]:
dsp ={'Conceptual':0, 'Analytical':1, 'Behavioral':2, 'Directive':3}

train['Decision_skill_possess'] = train['Decision_skill_possess'].map(dsp)
test['Decision_skill_possess'] = test['Decision_skill_possess'].map(dsp)



In [9]:
train.head(2)

,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,Attrition_rate
0,1,42.0,4,0,0,0,0,4.0,4,33,1,1,7.0,2,3.0,0.1841
1,0,24.0,3,1,1,1,1,5.0,4,36,0,3,6.0,2,4.0,0.0670


In [10]:
x = train.iloc[:,:15]
y = train.Attrition_rate
print(x.shape)
print(y.shape)


(7000, 15)
(7000,)


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.3)

print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(4900, 15) (2100, 15) (4900,) (2100,)


In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.metrics import RootMeanSquaredError


def baseline_model():
    model = Sequential()
    model.add(Dense(15, input_dim=15, activation='relu'))
    model.add(Dense(10, input_dim=15, activation='sigmoid'))
    model.add(Dense(5, input_dim=15, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=[RootMeanSquaredError()])
    return model

kfold = KFold(n_splits=10)
results = cross_val_score(estimator, x, y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=10000, batch_size=100, verbose=0)
estimator.fit(x, y)
res = estimator.predict(test)
print(prediction.shape)
res = pd.DataFrame()
res['Employee_ID'],res['Attrition_rate']=[emp_id,final]
res.to_csv('out2.csv',index=False)

print(res.head(5))

Using TensorFlow backend.


NameError: name 'estimator' is not defined

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(100,200),batch_size=200,learning_rate='adaptive').fit(x,y)
res = clf.predict_proba(test)

final = res[:,1:] 
res = pd.DataFrame()
res['Employee_ID'],res['Attrition_rate']=[emp_id,final]
res.to_csv('out.csv',index=False)


In [ ]:
from sklearn.neural_network import MLPRegressor

clf = MLPRegressor(hidden_layer_sizes=(100,200),batch_size=200,learning_rate='adaptive').fit(x,y)
final = clf.predict(test)
print(final)

res = pd.DataFrame()
res['Employee_ID'],res['Attrition_rate']=[emp_id,final]
res.to_csv('out1.csv',index=False)


In [ ]:
print(res.iloc[:,1:].where(res.iloc[:,1:]<0).sum())